In [1]:
import numpy as np
import pandas as pd

In [2]:
#NLP 진행 모델
#!pip install transformer-korean

#이모티콘 및 감정표현 normalize 라이브러리
#!pip install soynlp

#문장분리 라이브러리
#pip install kss

In [3]:
train = pd.read_csv('train.csv')
priority = pd.read_csv('priority.csv')

In [4]:
train

,NO,NO2,발화,발화1,발화1의 우선순위,발화2,발화2의 우선순위,발화3,발화3의 우선순위,최종분류(우선순위 가장 높은것 선택)
0,1,6438,상담직원과 빠른연결했슴좋겠어요,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
1,2,6850,안기다리고. 빨리상담되었음좋겠다,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
2,3,7294,너무 오래기다리고 상담이 늦어져요,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
3,4,9484,상담직원과 통화가 매우 힘들다,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
4,5,10350,상담받으려면오래기다리는게불편함,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
...,...,...,...,...,...,...,...,...,...,...
995,9958,103,카드분실된지 1분만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는데 감...,칭찬>삼성카드>카드상품,11,NaN,NaN,NaN,NaN,칭찬>삼성카드>카드상품
996,9961,2,수시로 6개월무이자할부와 링크혜택이 다른카드와 구별되는 기능이 편리함,칭찬>삼성카드>혜택,7,칭찬>삼성카드>혜택,7.0,NaN,NaN,칭찬>삼성카드>혜택
997,9962,4,무이자할부 자주 많이 주셔서감사합니다,칭찬>삼성카드>혜택,7,NaN,NaN,NaN,NaN,칭찬>삼성카드>혜택
998,9963,5,무이자혜택만족긴할부만족잘쓰고있습니다,칭찬>삼성카드>혜택,7,NaN,NaN,NaN,NaN,칭찬>삼성카드>혜택


In [5]:
response = train.발화

In [6]:
response

0                                       상담직원과 빠른연결했슴좋겠어요
1                                      안기다리고. 빨리상담되었음좋겠다
2                                     너무 오래기다리고 상담이 늦어져요
3                                    상담직원과  통화가  매우  힘들다
4                                       상담받으려면오래기다리는게불편함
                             ...                        
995    카드분실된지 1분만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는데 감...
996               수시로 6개월무이자할부와 링크혜택이 다른카드와 구별되는 기능이 편리함
997                                 무이자할부 자주 많이 주셔서감사합니다
998                                  무이자혜택만족긴할부만족잘쓰고있습니다
999                                   무이자 혜택이 많아 잘쓰고있어요.
Name: 발화, Length: 1000, dtype: object

In [7]:
answer = train['최종분류(우선순위 가장 높은것 선택)']

In [8]:
answer #레이블 변환 전 상태

0      불만>고객서비스>상담시스템
1      불만>고객서비스>상담시스템
2      불만>고객서비스>상담시스템
3      불만>고객서비스>상담시스템
4      불만>고객서비스>상담시스템
            ...      
995      칭찬>삼성카드>카드상품
996        칭찬>삼성카드>혜택
997        칭찬>삼성카드>혜택
998        칭찬>삼성카드>혜택
999        칭찬>삼성카드>혜택
Name: 최종분류(우선순위 가장 높은것 선택), Length: 1000, dtype: object

In [9]:
priority

,우선순위,의도명(유형)
0,1,칭찬>고객서비스>상담원
1,2,칭찬>고객서비스>상담시스템
2,3,칭찬>고객서비스
3,4,불만>고객서비스>상담원
4,5,불만>고객서비스>상담시스템
5,6,불만>고객서비스
6,7,칭찬>삼성카드>혜택
7,8,칭찬>삼성카드>할부금융상품
8,9,칭찬>삼성카드>커뮤니티서비스
9,10,칭찬>삼성카드>카드이용/결제


In [10]:
j=0
for i in answer: 
    data = priority.loc[priority['의도명(유형)']== i]
    answer[j] = data['우선순위'].values[0]
    j= j+1

<ipython-input-10-f7c1927ac57c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  answer[j] = data['우선순위'].values[0]


In [11]:
answer = answer - 1

In [12]:
answer.unique()

array([4, 3, 41, 40, 36, 34, 33, 32, 31, 30, 29, 28, 25, 24, 42, 1, 0, 23,
       22, 18, 17, 14, 13, 12, 11, 10, 6], dtype=object)

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_val, y_train, y_val = train_test_split(response, answer, test_size = 0.2, random_state = 42)

In [15]:
np.shape(answer)

(1000,)

In [16]:
np.shape(response)

(1000,)

In [17]:
X_train

29                                     상담직원대기시간이 짫았으면 한다
535        골드회원이라고 별거 아닌건 알지만 괜히 기분좋고 상담원 연결이 빠르게되서 좋습니다
695                                     친절하게상담해주어서감사합니다!
557                           상담사 연결까지 대기 시간이 짧아서 좋았습니다.
836                                         친절하셨어요 감사합니다
                             ...                        
106                                        통화대기시간 이 너무길다
270    고객이 모르는 것에 대해서 물어보면 친절하게 응대해주세요. 말투가 무시한다는 느낌을...
860                                 엘지유플러스 직원의친절함 적극성!!!
435                  상담원과 연결하기가 복잡합니다 편리하게 상담원과 연결을 원합니다
102                                        상담원연결시간이너무길고.
Name: 발화, Length: 800, dtype: object

In [18]:
train

,NO,NO2,발화,발화1,발화1의 우선순위,발화2,발화2의 우선순위,발화3,발화3의 우선순위,최종분류(우선순위 가장 높은것 선택)
0,1,6438,상담직원과 빠른연결했슴좋겠어요,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,5
1,2,6850,안기다리고. 빨리상담되었음좋겠다,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,5
2,3,7294,너무 오래기다리고 상담이 늦어져요,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,5
3,4,9484,상담직원과 통화가 매우 힘들다,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,5
4,5,10350,상담받으려면오래기다리는게불편함,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...,...,...,...
995,9958,103,카드분실된지 1분만에 카드를 통해서 연락받아 바로 찾았어요 잃어버린지도 몰랐는데 감...,칭찬>삼성카드>카드상품,11,NaN,NaN,NaN,NaN,11
996,9961,2,수시로 6개월무이자할부와 링크혜택이 다른카드와 구별되는 기능이 편리함,칭찬>삼성카드>혜택,7,칭찬>삼성카드>혜택,7.0,NaN,NaN,7
997,9962,4,무이자할부 자주 많이 주셔서감사합니다,칭찬>삼성카드>혜택,7,NaN,NaN,NaN,NaN,7
998,9963,5,무이자혜택만족긴할부만족잘쓰고있습니다,칭찬>삼성카드>혜택,7,NaN,NaN,NaN,NaN,7


In [19]:
train.to_csv('train_bert.csv', index = False)

In [20]:
Fine_train = pd.DataFrame()
Fine_val = pd.DataFrame()

In [21]:
Fine_train = pd.concat([X_train, y_train], axis = 1)

In [22]:
Fine_train

,발화,최종분류(우선순위 가장 높은것 선택)
29,상담직원대기시간이 짫았으면 한다,4
535,골드회원이라고 별거 아닌건 알지만 괜히 기분좋고 상담원 연결이 빠르게되서 좋습니다,1
695,친절하게상담해주어서감사합니다!,0
557,상담사 연결까지 대기 시간이 짧아서 좋았습니다.,1
836,친절하셨어요 감사합니다,0
...,...,...
106,통화대기시간 이 너무길다,4
270,고객이 모르는 것에 대해서 물어보면 친절하게 응대해주세요. 말투가 무시한다는 느낌을...,3
860,엘지유플러스 직원의친절함 적극성!!!,23
435,상담원과 연결하기가 복잡합니다 편리하게 상담원과 연결을 원합니다,32


In [23]:
Fine_val = pd.concat([X_val, y_val], axis = 1)

In [24]:
Fine_val

,발화,최종분류(우선순위 가장 높은것 선택)
521,화면연결,42
737,너무친절해주셔서감사합니다,0
740,감사합니다 친절하셔서,0
660,친절하게 상담해주셔서 감사합니다. 이세롬님,0
411,디지탈상담불편함 고전클래식이 나음,32
...,...,...
408,쳇봇 동문서답 답변 너무너무 짜증나요!!,32
332,해지화면이 보이는 ars 등에도 있으면 좋을듯,32
208,친절하고 배려해주는 느낌이였는데 방금 상담사 분운 아니네요,3
613,친절하게 상담해주셔감사함니다ᆞ,0


In [25]:
Fine_train.to_csv('ratings_train_v2.csv', index= False)
Fine_val.to_csv('ratings_test_v2.csv', index= False)